# Loading Packages

In [12]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision.models import vgg16
from torch.quantization import QuantStub, DeQuantStub
import torch.quantization
import torch.optim as optim
from torchinfo import summary
from tqdm import tqdm
from vgg_models import QuantizableVGG16

# Load the Model
- Then make sure to wrap it around the nn.DataParallel because the model was saved previously

In [21]:
# Load model
model = QuantizableVGG16(num_classes=100)

# Prepare for quantized loading
model.eval()  # Important for quantization
model.fuse_model()
model.qconfig = torch.quantization.get_default_qconfig("fbgemm")
torch.quantization.prepare(model, inplace=True)
torch.quantization.convert(model, inplace=True)

# Wrap in DataParallel to match saved state dict
model = torch.nn.DataParallel(model)

# Load the Quantized Weights
- A successful load should say that all keys matched

In [14]:
# Load quantized state_dict
state_dict = torch.load("quantized_vgg16_cifar100.pth", map_location='cpu')
model.load_state_dict(state_dict)

<All keys matched successfully>

# Download the Data

In [16]:
transform_train = transforms.Compose([
    transforms.Resize(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5071, 0.4865, 0.4409],
                         std=[0.2673, 0.2564, 0.2762])
])

transform_test = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5071, 0.4865, 0.4409],
                         std=[0.2673, 0.2564, 0.2762])
])

trainset = torchvision.datasets.CIFAR100(root='./data', train=True,
                                         download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,
                                          shuffle=True, num_workers=4)

testset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                        download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=128,
                                         shuffle=False, num_workers=4)

# Convert the Quantized Model

In [19]:
quantized_model = torch.quantization.convert(model, inplace=False)
quantized_model = quantized_model.module  # unwrap from DataParallel
quantized_model.eval()
quantized_model.to('cpu')  # <--- this fixes the RuntimeError

QuantizableVGG16(
  (quant): Quantize(scale=tensor([0.0309]), zero_point=tensor([61]), dtype=torch.quint8)
  (dequant): DeQuantize()
  (features): Sequential(
    (0): QuantizedConvReLU2d(3, 64, kernel_size=(3, 3), stride=(1, 1), scale=0.0859326645731926, zero_point=0, padding=(1, 1))
    (1): Identity()
    (2): QuantizedConvReLU2d(64, 64, kernel_size=(3, 3), stride=(1, 1), scale=0.14404772222042084, zero_point=0, padding=(1, 1))
    (3): Identity()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): QuantizedConvReLU2d(64, 128, kernel_size=(3, 3), stride=(1, 1), scale=0.23559726774692535, zero_point=0, padding=(1, 1))
    (6): Identity()
    (7): QuantizedConvReLU2d(128, 128, kernel_size=(3, 3), stride=(1, 1), scale=0.3782609701156616, zero_point=0, padding=(1, 1))
    (8): Identity()
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): QuantizedConvReLU2d(128, 256, kernel_size=(3, 3), stride=(1, 1), sc

# Evaluation

In [20]:
def evaluate(model, dataloader):
    model.eval()
    model.to('cpu')  # ensure model is on CPU

    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in tqdm(dataloader, desc="Evaluating"):
            inputs = inputs.cpu()
            labels = labels.cpu()

            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    acc = 100 * correct / total
    print(f"Accuracy of the quantized model: {acc:.2f}%")
    return acc


evaluate(quantized_model, testloader)

Evaluating: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 79/79 [07:29<00:00,  5.69s/it]

Accuracy of the quantized model: 69.33%


69.33